In [ ]:
# 02_feature_engineering.ipynb

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Load data
data = pd.read_csv('../data/processed/user_post_interactions.csv')

# Feature 1: TF-IDF vectors for post text
tfidf = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf.fit_transform(data['text'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
data = pd.concat([data.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

# Feature 2: Topic match (binary if user is interested in the post's topic)
# Load or simulate user interests (stored in dict format from earlier)
import ast
user_interests = {}

for uid in data['user_id'].unique():
    user_interests[uid] = np.random.choice(['tech', 'food', 'travel', 'fitness', 'gaming', 'fashion', 'news'], 
                                           size=np.random.randint(1, 4), replace=False).tolist()

data['topic_match'] = data.apply(lambda row: int(row['topic'] in user_interests[row['user_id']]), axis=1)

# Feature 3: Interaction type one-hot encoding (optional)
interaction_ohe = pd.get_dummies(data['interaction'], prefix='interact')
data = pd.concat([data, interaction_ohe], axis=1)

# Drop unused columns
features = data.drop(columns=['interaction', 'text', 'post_id', 'user_id'])  # keep label
labels = data['label']

# Save features and labels
features.to_csv('../data/processed/features.csv', index=False)
labels.to_csv('../data/processed/labels.csv', index=False)

print("✅ Feature engineering complete. Saved to processed/")
